# TODO

## Major
- [x] create term-by-document matrix (calculate words frequncies for each term-document pair)
 - [ ] check that it's actually correct - especially if we don't map terms to wrong documents
- [x] convert term-by-document frequencies to tf-idf (calcualte tf-idf for each term-document pair)
 - [ ] check
- [ ] we may need actual (numpy?) matrix?
- [ ] LSI magic

### Minor
- [x] remove numbers from terms - done but not sure if it's good thing to do, maybe it's also important for relevancy of docs,
like for example when there is year written?

In [2]:
import pandas as pd
import numpy as np
import string
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
np.random.seed(42)

In [4]:
bp_data = pd.read_csv("articles.csv", header=0)

In [5]:
bp_data.head(1)

,author,claps,reading_time,link,title,text
0,Justin Lee,8.3K,11,https://medium.com/swlh/chatbots-were-the-next...,Chatbots were the next big thing: what happene...,"Oh, how the headlines blared:\nChatbots were T..."


In [6]:
def preprocess_docs(docs, use_lemmatizer = True):
    '''Tokenize and preprocess documents
    
    Parameters
    ----------
    use_lemmatizer : bool
                     Uses lemmazizer if True, othrerwise uses stemmer.
    '''
    preproccessed_docs = []
    
    # English stop words list
    en_stop = set(stopwords.words('english'))
    
    # Word tokenizer that removes punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    
    # lemmatizer / Stemmer
    if use_lemmatizer:
        lemmatizer = WordNetLemmatizer()
    else:
        stemmer = SnowballStemmer("english")
    
    for row in docs.itertuples(index=True, name='Doc'):
        text = row.text
        
        # remove numbers
        text = re.sub(r'\d+', '', text)
        
        text_words = tokenizer.tokenize(text)
        
        if use_lemmatizer:
            text_words = [lemmatizer.lemmatize(word, pos="v").lower() for word in text_words
                          if word not in string.punctuation and word.lower() not in en_stop]
        else:
            text_words = [stemmer.stem(word).lower() for word in text_words
                         if word not in string.punctuation and word.lower() not in en_stop]
        
        preproccessed_docs.append({'words': text_words})
    
    pdocs = pd.DataFrame(preproccessed_docs)
    return pdocs

In [7]:
preproccessed_docs = preprocess_docs(bp_data)
display(preproccessed_docs)

,words
0,"[oh, headline, blare, chatbots, next, big, thi..."
1,"[ever, find, look, question, concept, syntax, ..."
2,"[machine, learn, increasingly, move, hand, des..."
3,"[understand, machine, learning, big, question,..."
4,"[want, learn, apply, artificial, intelligence,..."
...,...
332,"[click, share, article, linkedin, skip, part, ..."
333,"[opinions, deep, neural, network, machine, lea..."
334,"[everyone, remotely, tune, recent, progress, m..."
335,"[one, biggest, misconceptions, around, idea, d..."


In [8]:
def get_term_by_document_frequency(preprocessed_docs):
    document_by_term = {}
    
    for index, row in preprocessed_docs.iterrows():
        doc_id = index
        doc_words = row['words']
        
        # computed later, @TODO: move computation here and fix below or remove
#         document_by_term[doc_id] = {
#             'total_words': len(doc_words)
#         }
        document_by_term[doc_id] = {}
        
        for word in set(row['words']):
            document_by_term[doc_id][word] = doc_words.count(word)

    df = pd.DataFrame(document_by_term)
    
    return df

In [9]:
df_frequency = get_term_by_document_frequency(preproccessed_docs)

In [10]:
df_frequency

,0,1,2,3,4,5,6,7,8,9,...,327,328,329,330,331,332,333,334,335,336
elements,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
developers,3.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
eventually,1.0,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
faster,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,1.0,2.0,NaN,1.0,NaN,NaN,NaN
arcane,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
loopy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
intelligible,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
intangible,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
oncologist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [64]:
def reduce_terms(df_frequency, max_df=1, min_df=0, max_terms=None):
    '''Remove unimportant terms from term-by-document matrix.
    
    Parameters
    ----------
    df : pd.DataFrame
    max_df : float , between [0, 1]
             Terms that appear in more % of documents will be ignored
    min_df : float , between [0, 1]
             Terms that appear in less % of documents will be ignored
    max_terms : int , None
                If not None, only top `max_terms` terms will be returned.
    '''
    df = df_frequency.copy()
    corpus_size = df.shape[1]

    if 'doc_frequency' not in df:
        df['doc_frequency'] = df_frequency.fillna(0).astype(bool).sum(axis=1) / corpus_size
            
    df = df[df.doc_frequency <= max_df]
    df = df[df.doc_frequency >= min_df]
    
    if max_terms is not None and max_terms < df.shape[0]:
        df['term_count'] = df_frequency.fillna(0).sum(axis=1)
        df = df.sort_values('term_count', ascending=False)
        df = df.head(max_terms)
        df.drop('term_count',axis=1, inplace=True)
    
    return df

In [65]:
reduce_terms(df_frequency).sort_values('doc_frequency', ascending=False).shape

(16603, 338)

In [66]:
reduce_terms(df_frequency, 0.8, 0.1, 1000).sort_values('doc_frequency', ascending=False)

,0,1,2,3,4,5,6,7,8,9,...,328,329,330,331,332,333,334,335,336,doc_frequency
take,7.0,3.0,4.0,5.0,9.0,2.0,NaN,4.0,NaN,5.0,...,5.0,2.0,NaN,6.0,2.0,1.0,4.0,NaN,3.0,0.777448
way,9.0,1.0,1.0,2.0,4.0,4.0,2.0,2.0,2.0,9.0,...,NaN,11.0,NaN,5.0,1.0,1.0,NaN,1.0,7.0,0.765579
machine,6.0,2.0,10.0,11.0,2.0,NaN,20.0,3.0,2.0,5.0,...,2.0,8.0,2.0,1.0,6.0,11.0,4.0,4.0,NaN,0.762611
find,NaN,3.0,2.0,3.0,1.0,3.0,1.0,2.0,1.0,7.0,...,3.0,5.0,NaN,4.0,3.0,2.0,NaN,NaN,5.0,0.762611
go,2.0,2.0,2.0,3.0,4.0,2.0,1.0,2.0,NaN,10.0,...,1.0,2.0,3.0,NaN,8.0,5.0,NaN,2.0,2.0,0.756677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
devices,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.100890
straight,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.100890
body,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.100890
win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.100890


In [67]:
df_reduced = reduce_terms(df_frequency, 0.8, 0.1)

In [68]:
def get_tf_idf(df_frequency):
    df = df_frequency.copy()
    # tf := word frequency / total frequency
    df.loc['total_words'] = df.sum()
        
    df = df.drop('total_words')[:] / df.loc['total_words']
    
    # idf := log ( len(all_documents) / len(documents_containing_word) )
    
    corpus_size = df.shape[1]

    # number of non-zero cols
    if 'doc_frequency' not in df_frequency:
        df['doc_frequency'] = df.fillna(0).astype(bool).sum(axis=1)
        
    df['idf'] = np.log( corpus_size / df['doc_frequency'] )
    
    # tf-idf := tf * idf
    _cols = df.columns.difference(['idf', 'doc_frequency'])
    df[_cols] = df[_cols].multiply(df["idf"], axis="index")
    
    df.drop(columns=['doc_frequency', 'idf'], inplace=True)
    
    return df

In [69]:
df_tf_idf = get_tf_idf(df_reduced)
display(df_tf_idf)

,0,1,2,3,4,5,6,7,8,9,...,327,328,329,330,331,332,333,334,335,336
eventually,0.018060,NaN,NaN,NaN,0.014865,0.011133,NaN,0.038043,NaN,NaN,...,0.020060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
faster,0.054344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.036486,0.050381,0.045575,NaN,0.018089,NaN,NaN,NaN
new,0.032927,0.051526,0.17716,0.192165,0.040655,0.030447,0.120535,NaN,0.110586,0.010705,...,0.042670,0.077119,0.049740,0.091579,0.062131,0.060631,0.131524,NaN,0.116928,0.016557
long,0.017310,0.054175,NaN,NaN,NaN,NaN,NaN,0.036464,NaN,0.033767,...,NaN,NaN,NaN,NaN,0.021775,NaN,0.017286,NaN,0.030735,NaN
quality,0.018382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.102250,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
behavior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.051627,0.202388,NaN
neuron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.007021,NaN,NaN,NaN,NaN,NaN,NaN,0.154598,0.370367,0.114420
improvements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.038642,NaN,NaN,NaN,NaN,NaN,0.068129,NaN


In [70]:
values = df_tf_idf.fillna(0).to_numpy()
values

array([[0.01805959, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.05434402, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.03292718, 0.0515256 , 0.17715982, ..., 0.        , 0.11692843,
        0.0165566 ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.15459806, 0.3703671 ,
        0.11442003],
       [0.        , 0.        , 0.        , ..., 0.        , 0.06812862,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [71]:
u, s, vh = np.linalg.svd(values, full_matrices=True)

In [19]:
vh

array([[ 4.38027008e-02,  4.52527257e-02,  5.49950587e-02, ...,
         5.03892281e-02,  5.13812996e-02,  5.13567737e-02],
       [-1.20471112e-02, -9.93265786e-03, -8.84456061e-03, ...,
        -1.38319783e-02, -1.34522571e-02, -1.30991734e-02],
       [ 4.57748185e-03,  7.51473102e-03,  1.46184934e-02, ...,
         1.57943086e-02,  1.00991313e-02,  1.59776393e-02],
       ...,
       [ 0.00000000e+00, -4.17757167e-17, -1.17964460e-16, ...,
         1.26634814e-16,  2.42861287e-17,  3.46944695e-17],
       [ 0.00000000e+00, -2.71584398e-16,  1.10813035e-16, ...,
         6.41847686e-17,  1.04083409e-17,  4.51028104e-17],
       [ 8.16485257e-01,  2.22044605e-16,  1.66533454e-16, ...,
         4.80598024e-17, -4.01561380e-17, -2.02745806e-17]])